# Reports
1. Inputs: data with feature engineering 
2. Outputs: prepared data to ml modeling
3. ...

# 1.0 Imports

###### 1.1 Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

from xgboost import XGBRFClassifier

from boruta import BorutaPy

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

import pickle

import time
import warnings

# Tempo de processamento
time_agora = time.time()

###### 1.2 Colecting data

In [2]:
df = pickle.load(open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\df_to_data_preparation.pkl', 'rb'))

# 6.0 Data Preparation

### 6.1 Preprocessing

In [3]:
# backup
df_ = df.copy()

In [4]:
df.columns

Index(['id', 'Gender', 'Age', 'Driving_License', 'Region_Code',
       'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium',
       'Policy_Sales_Channel', 'Vintage', 'Response', 'Gender_',
       'Vehicle_Damage_', 'Vehicle_Age_', 'Policy_Sales_Channel_Response_Mean',
       'Region_Code_Response_Mean', 'High_Response_Age', 'Log_Annual_Premium'],
      dtype='object')

In [5]:
# backup
df = df_.copy()

# Feature dummies
temp = pd.get_dummies(df['Vehicle_Age_'], prefix='Vehicle_Age')
temp.drop('Vehicle_Age_0', axis=1, inplace=True)
df = pd.concat([df, temp], axis=1)

# Removendo features que não seguirão para modelagem
features_remover = ['id', 'Gender', 'Vehicle_Age', 'Vehicle_Age_', 'Vehicle_Damage', 'Policy_Sales_Channel', 
                    'Region_Code', 'Vintage', 'Annual_Premium']
df.drop(features_remover, axis=1, inplace=True)
model_columns = df.drop('Response', axis=1).columns.to_list()
print(model_columns)

# target
target = ['Response']

# Split X and y
train = df.loc[df['Response'].notnull()]
test = df.loc[df['Response'].isnull()]
X = train.drop(target, axis=1)
X = np.array(X)
y = train[target]
y = np.array(y).reshape(1, len(y))[0]

# Split train end test
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

# Split train_ end valid
X_train, X_valid, y_train, y_valid = train_test_split(X_, y_, test_size=0.3, random_state=101)

# Standard Scaler fit
X_scaler = StandardScaler().fit(X_train)

# Scaler transform
X = X_scaler.transform(X)
X_ = X_scaler.transform(X_)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)
X_valid = X_scaler.transform(X_valid)

# Resolvendo desbalanceamento
# Nearmiss
near = NearMiss()
X_near, y_near = near.fit_resample(X_, y_)
X_train_near, y_train_near = near.fit_resample(X_train, y_train)

# Smote
smote = SMOTE()
X_smote, y_smote = smote.fit_resample(X_, y_)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

['Age', 'Driving_License', 'Previously_Insured', 'Gender_', 'Vehicle_Damage_', 'Policy_Sales_Channel_Response_Mean', 'Region_Code_Response_Mean', 'High_Response_Age', 'Log_Annual_Premium', 'Vehicle_Age_1', 'Vehicle_Age_2']


### 6.2 Feature Selection

##### 6.2.1 Principais Insights:
1. As features com maior relevância identificadas através de Exploração de Dados foram: Vehicle_Damage, Previously_Insured, Vehicle_Age, Policy_Sales_Channels_Response_Mean, Age
2. 6 Dimensões compõem 95% de toda a variabilidade das features.
3. A feature Driving_License não agrega ao modelo

##### 6.2.2 Por Correlação

In [6]:
correlations = abs(train.corr()['Response'].drop('Response', axis=0))
correlations
ranking = pd.DataFrame()
ranking['correlations'] = correlations

##### 6.2.3 RFE (Ranking por eliminação recursiva)

In [7]:
# model description
xgbrf_model = XGBRFClassifier(use_label_encoder=False, eval_metric='error')

# rfe description
rfe_model = RFE(xgbrf_model)

# fiting model
rfe_fit = rfe_model.fit(X_train_near, y_train_near)

# ranking
ranking['rfe_xgb'] = rfe_fit.ranking_
print(rfe_fit.ranking_)
ranking.sort_values('rfe_xgb').index.to_list()

[1 7 6 5 3 1 1 4 2 1 1]


['Age',
 'Policy_Sales_Channel_Response_Mean',
 'Region_Code_Response_Mean',
 'Vehicle_Age_1',
 'Vehicle_Age_2',
 'Log_Annual_Premium',
 'Vehicle_Damage_',
 'High_Response_Age',
 'Gender_',
 'Previously_Insured',
 'Driving_License']

Alta preferência por features numéricas, não confere com a exploração de dados, possivelmente overfitado.

##### 6.2.4 Importances

###### 6.2.4.1 Random Forest

In [8]:
# model definition
rf_model = RandomForestClassifier(n_estimators=100, min_samples_leaf=1000, random_state=101)

# fiting model
rf_fit = rf_model.fit(X_train_smote, y_train_smote)

# preditions
rf_predict = rf_fit.predict(X_valid)
ranking['importance_rf'] = rf_model.feature_importances_

# metrics
print(classification_report(y_valid, rf_predict))

              precision    recall  f1-score   support

         0.0       0.98      0.67      0.80     80152
         1.0       0.28      0.93      0.43     11315

    accuracy                           0.70     91467
   macro avg       0.63      0.80      0.62     91467
weighted avg       0.90      0.70      0.75     91467



###### 6.2.4.2 XGRFBoost

In [9]:
# fiting model
xgbrf_fit = xgbrf_model.fit(X_train_smote, y_train_smote)

# predictions
xgbrf_predict = xgbrf_fit.predict(X_valid)
ranking['importance_xgb'] = xgbrf_model.feature_importances_

# metrics
print(classification_report(y_valid, xgbrf_predict))

              precision    recall  f1-score   support

         0.0       0.99      0.66      0.79     80152
         1.0       0.28      0.93      0.43     11315

    accuracy                           0.70     91467
   macro avg       0.63      0.80      0.61     91467
weighted avg       0.90      0.70      0.75     91467



##### 6.2.5 Boruta

In [ ]:
# model descrition
boruta_model = BorutaPy(xgbrf_model, random_state=101)

# fiting model
boruta_fit = boruta_model.fit(X_train_near, y_train_near)

# ranking features
ranking['boruta'] = boruta_fit.ranking_
print(boruta_fit.ranking_)
ranking.sort_values('boruta', ascending=False).index.to_list()

In [ ]:
# ranking_boruta = [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# ranking['boruta'] = ranking_boruta

##### 6.2.6 Comparison

In [ ]:
ranking

1. As colunas "correlations", "importance_rf" e "importance_xgb" são quantitativas e estão na mesma escala
2. A coluna "rfe_xgb" é tipo tipo ranking, quanto menor o número melhor a colocação
3. A coluna "boruta" é binária, 1 significa uma feature relevante e 2 uma feature a ser dispensada

In [ ]:
ranking_1 = ranking[['correlations', 'importance_rf', 'importance_xgb']]

ranking_1.sort_values('importance_xgb', ascending=False, inplace=True)

ranking_1.plot(kind='bar', stacked=True)
plt.title('Importancias das features');

##### 6.2.7 Removing expendable features

In [ ]:
X = np.delete(X, 1, 1)
X_smote = np.delete(X_smote, 1, 1)
X_near = np.delete(X_near, 1, 1)
X_train_smote = np.delete(X_train_smote, 1, 1)
X_train_near = np.delete(X_train_near, 1, 1)
X_valid = np.delete(X_valid, 1, 1)
X_test = np.delete(X_test, 1, 1)

model_columns.pop(1)

# 7.0 Exporting data

###### 7.1 Scaler

In [ ]:
pickle.dump(X_scaler, open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\webapp\\scaler_standard_scaler.pkl', 'wb'))

###### 7.2 Arrays

In [ ]:
X_to_modeling = [X_valid, X_test, X]
X_imb_to_modeling = [X_smote, X_near, X_train_smote, X_train_near]
y_to_modeling = [y_smote, y_near, y_train_smote, y_train_near, y_valid, y_test, y]
pickle.dump(X_to_modeling, open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\X_to_modeling.pkl', 'wb'))
pickle.dump(X_imb_to_modeling, open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\X_imb_to_modeling.pkl', 'wb'))
pickle.dump(y_to_modeling, open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\y_to_modeling.pkl', 'wb'))

###### 7.4 Columns

In [ ]:
pickle.dump(model_columns, open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\model_columns.pkl', 'wb'))

###### 7.5 Ranking of features

In [ ]:
pickle.dump(ranking, open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\insurance_sales_predict\\exportings\\ranking_feature_selection.pkl', 'wb'))

###### 7.5 Expendable features

In [ ]:
pickle.dump(features_remover, open('D:\\My Drive\\Pessoal\\Projetos\\insurance_sales_predict\\webapp\\_features_remover.pkl', 'wb'))

# Time

In [ ]:
print(f'O tempo de processamento do projeto foi de: {int(round(time.time()-time_agora, 2)/60)} minutos')